In [68]:
pip install skyfield

  Created wheel for skyfield: filename=skyfield-1.39-py3-none-any.whl size=417566 sha256=5a16c5486e0da844c33a92777ad7393eee5b76fee7d4cd0a309eb86f254d7b2e
  Stored in directory: c:\users\izzat\appdata\local\pip\cache\wheels\88\e4\82\79702c7fd642ef81355df5ad2a355db0a5d42744b42533801b
  Created wheel for jplephem: filename=jplephem-2.16-py3-none-any.whl size=46775 sha256=142ae1d6f4e3bce05af793fa32c3cf90f22f79779031387698e1589f8b101a88
  Stored in directory: c:\users\izzat\appdata\local\pip\cache\wheels\32\6c\56\475357f9ee778f1995ba2be80c79e0fdded7162e1dc3ddc578
Successfully built skyfield jplephem
Note: you may need to restart the kernel to use updated packages.


In [50]:
pip install pytz

Note: you may need to restart the kernel to use updated packages.


In [219]:
from skyfield.api import *
from pytz import timezone
import datetime as dt
from skyfield import almanac

#set zone using pytz
zone = timezone('Asia/Kuala_Lumpur')
maghrib_list = []
date = []
zuhr_list = []
syuruk_list = []
for i in range(1,32):
    # localize naive datetime (with no timezone)
    now = zone.localize(dt.datetime(2021, 8, i))
    # set midnight time based on current time (now) <- datetime
    midnight = now.replace(hour = 0, minute = 0, second = 0, microsecond = 0)
    # set the next midnight <- datetime
    next_midnight = midnight + dt.timedelta(days =1)

    #load tmescale for all date and time
    ts = load.timescale()
    #returns timescale for midnight from the python datetime
    t0 = ts.from_datetime(midnight)
    #returns timescale for next midnight from python datetime
    t1 = ts.from_datetime(next_midnight)
    #load ephemeris
    eph = load('de440s.bsp')
    #set topocentric location based on the wgs84
    penang = wgs84.latlon(5.33*N, 100.2265*E)

    # find time when a body transits the meridian
    f = almanac.meridian_transits(eph, eph['Sun'], penang)
    g = almanac.sunrise_sunset(eph, penang)
    # set the times and events
    zuhrtimes, zuhrevents = almanac.find_discrete(t0, t1, f)
    maghribtimes, maghribevents = almanac.find_discrete(t0, t1, g)
    
    #set times for the respective events
    zuhr = zuhrtimes[zuhrevents == 1]
    maghrib = maghribtimes[maghribevents == 0]
    syuruk = maghribtimes[maghribevents == 1]

    t = zuhr[0]
    t2 = maghrib[0]
    t3 = syuruk[0]
    tstr = str(t.astimezone(zone))[:11]
    tstr1 = str(t.astimezone(zone) + dt.timedelta(minutes =1, seconds = 4))[11:19]
    tstr2 = str(t2.astimezone(zone))[11:19]
    tstr3 = str(t3.astimezone(zone))[11:19]
    date.append(tstr)
    maghrib_list.append(tstr2)
    zuhr_list.append(tstr1)
    syuruk_list.append(tstr3)


In [177]:
import pandas as pd

# Print table
waktu_solat = pd.DataFrame(list(zip(date, syuruk_list, zuhr_list, maghrib_list)), columns = ['Date', 'Syuruk', 'Zuhr', 'Maghrib'] )
waktu_solat

,Date,Syuruk,Zuhr,Maghrib
0,2021-08-01,07:14:58,13:26:30,19:35:49
1,2021-08-02,07:15:01,13:26:26,19:35:38
2,2021-08-03,07:15:03,13:26:21,19:35:27
3,2021-08-04,07:15:05,13:26:16,19:35:15
4,2021-08-05,07:15:06,13:26:10,19:35:02
5,2021-08-06,07:15:07,13:26:03,19:34:48
6,2021-08-07,07:15:07,13:25:56,19:34:34
7,2021-08-08,07:15:07,13:25:49,19:34:19
8,2021-08-09,07:15:06,13:25:41,19:34:03
9,2021-08-10,07:15:05,13:25:32,19:33:47


In [12]:
from skyfield import *
from skyfield.api import *
from pytz import timezone
import datetime as dt
from skyfield import almanac
from astropy import units as u

planets = load('de421.bsp')
bumi, bulan, matahari = planets['earth'], planets['moon'],planets['sun']
eph = api.load('de421.bsp')
zone = timezone('Asia/Kuala_Lumpur')
pulau_pinang = bumi + wgs84.latlon(5.33*N, 100.2265*E)
penang = wgs84.latlon(5.33*N, 100.2265*E)
tarikh = []
altitud = []
jarak_lengkung_list = []
bulan_baru = []
fasa_bulan = []
maghrib_list = []
hijri_hari = 17
hijri_bulan = 5
list_hijri_hari = [hijri_hari]
list_hijri_bulan = [hijri_bulan]
for i in range(1,13):
    for f in range (1,32):
        if (i in [2,4,6,9,11] and f >30) or (i == 2 and f > 28):
            continue
        now = zone.localize(dt.datetime(2021, i, f))
        midnight = now.replace(hour = 0, minute = 0, second = 0, microsecond = 0)
        next_midnight = midnight + dt.timedelta(days =1)
        ts = load.timescale()
        t0 = ts.from_datetime(midnight)
        t1 = ts.from_datetime(next_midnight)
        eph = load('de440s.bsp')
        g = almanac.sunrise_sunset(eph, penang)
        maghribtimes, maghribevents = almanac.find_discrete(t0, t1, g)
        maghrib = maghribtimes[maghribevents == 0]
        lok_bulan = pulau_pinang.at(maghrib[0]).observe(bulan)
        lok_matahari = pulau_pinang.at(maghrib[0]).observe(matahari)
        jarak_lengkung = lok_matahari.separation_from(lok_bulan)
        alt_bulan, az_bulan, jarak_bulan = lok_bulan.apparent().altaz()
        phase = almanac.moon_phase(eph, maghrib)
        t7 = str(maghrib[0].astimezone(zone))[:11]
        tarikh.append(t7)
        t2 = '%.4f' %alt_bulan.degrees
        altitud.append(t2)
    #     print('%.4f' %az_bulan.degrees)
        t3 = '%.4f' %jarak_lengkung.degrees
        jarak_lengkung_list.append(t3)
        t6 = '%.4f' %phase.degrees
        fasa_bulan.append(t6)
        t4 = float(jarak_lengkung.degrees)
        t5 = float(alt_bulan.degrees)
        t_maghrib = maghrib[0]
        tstr2 = str(t_maghrib.astimezone(zone))[11:19]  
        maghrib_list.append(tstr2)      
        if hijri_hari < 29 and hijri_bulan<13:
            hijri_hari += 1
            list_hijri_hari.append(hijri_hari)
            list_hijri_bulan.append(hijri_bulan)
            bulan_baru.append('N/A')
            continue     
        elif hijri_hari == 30 and hijri_bulan != 12:
            hijri_bulan += 1
            hijri_hari = 1
            list_hijri_hari.append(hijri_hari)
            list_hijri_bulan.append(hijri_bulan)
            bulan_baru.append('N/A')
            continue
        elif hijri_hari == 29 and hijri_bulan<13:
            if float(phase.degrees) < 17:
                if t4 < 6.4:
                    bulan_baru.append('Tak mencukupi syarat')
                    hijri_hari += 1
                    list_hijri_hari.append(hijri_hari)
                    list_hijri_bulan.append(hijri_bulan)
                    continue
                elif t4 > 6.4 and t5 < 3:
                    bulan_baru.append('Tak mencukupi syarat')
                    hijri_hari += 1
                    list_hijri_hari.append(hijri_hari)
                    continue
                else:
                    bulan_baru.append('Mencukupi syarat 6.4 & 3')
                    if hijri_bulan == 12:
                        hijri_hari = 1
                        hijri_bulan = 1
                        list_hijri_hari.append(hijri_hari)
                        list_hijri_bulan.append(hijri_bulan)
                        continue
                    else:
                        hijri_bulan += 1
                        hijri_hari = 1                        
                        list_hijri_hari.append(hijri_hari)
                        list_hijri_bulan.append(hijri_bulan)                        
                        continue
            else:
                bulan_baru.append('N/A')
                hijri_hari += 1
                list_hijri_hari.append(hijri_hari)
                list_hijri_bulan.append(hijri_bulan)
                continue
        elif hijri_hari == 30 and hijri_bulan == 12:
            hijri_hari = 1
            hijri_bulan = 1
            list_hijri_hari.append(hijri_hari)
            list_hijri_bulan.append(hijri_bulan)
            continue

    
        
    
# print(distance)

In [13]:
import pandas as pd

# Print table
takwim = pd.DataFrame(list(zip(tarikh, altitud, jarak_lengkung_list, fasa_bulan, bulan_baru, list_hijri_hari, list_hijri_bulan, maghrib_list)), columns = ['Tarikh Masihi', 'Altitud', 'Jarak Lengkung', 'Fasa Bulan', 'Syarat', 'Hijri/hari','Hijri/bulan', 'Maghrib'] )
takwim

,Tarikh Masihi,Altitud,Jarak Lengkung,Fasa Bulan,Syarat,Hijri/hari,Hijri/bulan,Maghrib
0,2021-01-01,-27.5422,151.2701,207.6487,N/A,17,5,19:17:19
1,2021-01-02,-39.6562,139.1356,219.9207,N/A,18,5,19:17:49
2,2021-01-03,-51.9196,126.8451,232.4023,N/A,19,5,19:18:19
3,2021-01-04,-64.3092,114.4058,245.0763,N/A,20,5,19:18:49
4,2021-01-05,-76.7400,101.8175,257.9252,N/A,21,5,19:19:18
...,...,...,...,...,...,...,...,...
360,2021-12-27,-82.2596,85.5018,274.5738,N/A,22,5,19:14:37
361,2021-12-28,-72.9563,73.0563,287.2426,N/A,23,5,19:15:08
362,2021-12-29,-60.7432,60.1980,300.3033,N/A,24,5,19:15:38
363,2021-12-30,-47.6982,46.9298,313.7499,N/A,25,5,19:16:09


In [14]:
takwim.to_csv('takwim1.csv')

In [321]:
def takwim_syarat_baru(tahun):
    for i in range(1,13):
        for f in range (1,32):
            if (i in [2,4,6,9,11] and f >30) or (i == 2 and f > 28):
                continue
            now = zone.localize(dt.datetime(tahun, i, f))

            midnight = now.replace(hour = 0, minute = 0, second = 0, microsecond = 0)
            next_midnight = midnight + dt.timedelta(days =1)
            ts = load.timescale()
            t0 = ts.from_datetime(midnight)
            t1 = ts.from_datetime(next_midnight)
            eph = load('de440s.bsp')
            g = almanac.sunrise_sunset(eph, penang)
            maghribtimes, maghribevents = almanac.find_discrete(t0, t1, g)
            maghrib = maghribtimes[maghribevents == 0]
            lok_bulan = pulau_pinang.at(maghrib[0]).observe(bulan)
            lok_matahari = pulau_pinang.at(maghrib[0]).observe(matahari)
            jarak_lengkung = lok_matahari.separation_from(lok_bulan)
            alt_bulan, az_bulan, jarak_bulan = lok_bulan.apparent().altaz()
            phase = almanac.moon_phase(eph, maghrib)
            t1 = str(maghrib[0].astimezone(zone))[:11]
            tarikh.append(t1)
            t2 = '%.4f' %alt_bulan.degrees
            altitud.append(t2)
        #     print('%.4f' %az_bulan.degrees)
            t3 = '%.4f' %jarak_lengkung.degrees
            jarak_lengkung_list.append(t3)
            t6 = '%.4f' %phase.degrees
            fasa_bulan.append(t6)
            t4 = float(jarak_lengkung.degrees)
            t5 = float(alt_bulan.degrees)
            if float(phase.degrees) < 14:
                if t4 < 6.4:
                    bulan_baru.append('Tak mencukupi syarat')
                elif t4 > 6.4 and t5 < 3:
                    bulan_baru.append('Tak mencukupi syarat')  
                else:
                    bulan_baru.append('Mencukupi syarat 6.4 & 3')
            else:
                bulan_baru.append('N/A')

In [348]:
def kalendar_hijri(phase, hari, bulan):
    list_hijri_hari = [hari]
    list_hijri_bulan = [bulan]
    if hijri_hari < 29 and hijri_bulan<13:
            hijri_hari += 1
            list_hijri_hari.append(hijri_hari)
            list_hijri_bulan.append(hijri_bulan)
            bulan_baru.append('N/A')
            continue     
        elif hijri_hari == 30 and hijri_bulan != 12:
            hijri_bulan += 1
            hijri_hari = 1
            list_hijri_hari.append(hijri_hari)
            list_hijri_bulan.append(hijri_bulan)
            bulan_baru.append('N/A')
            continue
        elif hijri_hari == 29 and hijri_bulan<13:
            if float(phase.degrees) < 17:
                if t4 < 6.4:
                    bulan_baru.append('Tak mencukupi syarat')
                    hijri_hari += 1
                    list_hijri_hari.append(hijri_hari)
                    list_hijri_bulan.append(hijri_bulan)
                    continue
                elif t4 > 6.4 and t5 < 3:
                    bulan_baru.append('Tak mencukupi syarat')
                    hijri_hari += 1
                    list_hijri_hari.append(hijri_hari)
                    continue
                else:
                    bulan_baru.append('Mencukupi syarat 6.4 & 3')
                    if hijri_bulan == 12:
                        hijri_hari = 1
                        hijri_bulan = 1
                        list_hijri_hari.append(hijri_hari)
                        list_hijri_bulan.append(hijri_bulan)
                        continue
                    else:
                        hijri_bulan += 1
                        hijri_hari = 1                        
                        list_hijri_hari.append(hijri_hari)
                        list_hijri_bulan.append(hijri_bulan)                        
                        continue
            else:
                bulan_baru.append('N/A')
                hijri_hari += 1
                list_hijri_hari.append(hijri_hari)
                list_hijri_bulan.append(hijri_bulan)
                continue
        elif hijri_hari == 30 and hijri_bulan == 12:
            hijri_hari = 1
            hijri_bulan = 1
            list_hijri_hari.append(hijri_hari)
            list_hijri_bulan.append(hijri_bulan)
            continue

        print(hijri)